### Why we only get data up to 2025 and not before?
### Why the deprecated warnings?

In [1]:
import pystac_client

catalog = pystac_client.Client.open(
    "https://stac.core.eopf.eodc.eu"
)

In [2]:
bbox_villaviciosa = [
    -5.432,   # min lon
    43.497,  # min lat
    -5.374,   # max lon
    43.534   # max lat
]

temporal_extent = ["2022-01-01", None] 

items = list(
    catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox_villaviciosa,
        datetime=temporal_extent,
        #query={"eo:cloud_cover": {"lt": 20}},
    ).items()
)
print(f"Found {len(items)} Sentinel-2 L2A products")
for item in items:
    print(item.datetime)

/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251104T112139_N0511_R037_T30TUP_20251104T134759' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251104T112139_N0511_R037_T29TQJ_20251104T134759' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2A_MSIL2A_20251101T112241_N0511_R037_T30TUP_20251101T151319' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2A_MSIL2A_20251101T112241_N0511_R037_T29TQJ_20251101T151319' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251101T111119_N0511_R137_T30TUP_20251101T

Found 194 Sentinel-2 L2A products
2026-02-02 11:21:59.024000+00:00
2026-02-02 11:21:59.024000+00:00
2026-01-30 11:12:09.025000+00:00
2026-01-30 11:12:09.025000+00:00
2026-01-28 11:23:21.025000+00:00
2026-01-28 11:23:21.025000+00:00
2026-01-25 11:13:41.025000+00:00
2026-01-25 11:13:41.025000+00:00
2026-01-23 11:22:49.024000+00:00
2026-01-23 11:22:49.024000+00:00
2026-01-18 11:24:11.025000+00:00
2026-01-18 11:24:11.025000+00:00
2026-01-15 11:14:21.025000+00:00
2026-01-15 11:14:21.025000+00:00
2026-01-13 11:23:29.024000+00:00
2026-01-13 11:23:29.024000+00:00
2026-01-10 11:21:31.024000+00:00
2026-01-10 11:21:31.024000+00:00
2026-01-10 11:13:39.024000+00:00
2026-01-10 11:13:39.024000+00:00
2026-01-08 11:24:51.025000+00:00
2026-01-03 11:23:59.024000+00:00
2025-12-31 11:13:59.024000+00:00
2025-12-31 11:13:59.024000+00:00
2025-12-29 11:25:11.025000+00:00
2025-12-28 11:15:21.024000+00:00
2025-12-26 11:15:01.025000+00:00
2025-12-21 11:25:21.024000+00:00
2025-12-21 11:13:59.024000+00:00
2025-12-1

/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20250405T110619_N0511_R137_T30TUP_20250405T140215' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20250405T110619_N0511_R137_T29TQJ_20250405T140215' is deprecated.
  warnings.warn(


### Also when loading STAC items into xarray, some seem to be actually missing

In [11]:
import xarray as xr
from pyproj import Transformer

datasets, included, skipped, crs_list = [], [], [], []

for item in items:
    url = item.assets["product"].href

    # --- Try to open dataset ---
    try:
        ds = xr.open_dataset(url, engine="eopf-zarr", chunks={})
    except Exception as e:
        print(f"\n-{e}")
        skipped.append(url)
        continue

    # --- Get CRS ---
    crs = ds.rio.crs
    print(f"\n-{url.split('/')[-1]}")
    print(f"   CRS: {crs}")

    if crs is None:
        print("   No CRS found, skipping")
        skipped.append(url)
        continue

    # --- Reproject WGS84 bbox to dataset CRS ---
    transformer = Transformer.from_crs("EPSG:4326", crs, always_xy=True)
    minx, miny = transformer.transform(bbox_villaviciosa[0], bbox_villaviciosa[1])
    maxx, maxy = transformer.transform(bbox_villaviciosa[2], bbox_villaviciosa[3])

    # --- Handle Y axis direction ---
    y_slice = slice(maxy, miny) if ds.y[0] > ds.y[-1] else slice(miny, maxy)

    # --- Clip ---
    ds_clipped = ds.sel(x=slice(minx, maxx), y=y_slice)

    if ds_clipped.sizes["x"] == 0 or ds_clipped.sizes["y"] == 0:
        print("   No overlap with bbox after reprojection")
        skipped.append(url)
        continue

    print(f"   Clipped size: {dict(ds_clipped.sizes)}")

    # --- Store dataset ---
    datasets.append(ds_clipped)
    included.append(url)
    crs_list.append(crs)

print(f"\n✓ Included: {len(included)} | ✗ Skipped: {len(skipped)}")


-S2B_MSIL2A_20260202T112159_N0511_R037_T30TUP_20260202T151511.zarr
   CRS: EPSG:32630
   Clipped size: {'x': 481, 'y': 398}

-S2B_MSIL2A_20260202T112159_N0511_R037_T29TQJ_20260202T151511.zarr
   CRS: EPSG:32629
   Clipped size: {'x': 451, 'y': 432}

-S2B_MSIL2A_20260130T111209_N0511_R137_T30TUP_20260130T150430.zarr
   CRS: EPSG:32630
   Clipped size: {'x': 481, 'y': 398}

-S2B_MSIL2A_20260130T111209_N0511_R137_T29TQJ_20260130T150430.zarr
   CRS: EPSG:32629
   Clipped size: {'x': 451, 'y': 432}

-S2C_MSIL2A_20260128T112321_N0511_R037_T30TUP_20260128T151411.zarr
   CRS: EPSG:32630
   Clipped size: {'x': 481, 'y': 398}

-S2C_MSIL2A_20260128T112321_N0511_R037_T29TQJ_20260128T151411.zarr
   CRS: EPSG:32629
   Clipped size: {'x': 451, 'y': 432}

-S2C_MSIL2A_20260125T111341_N0511_R137_T30TUP_20260125T145411.zarr
   CRS: EPSG:32630
   Clipped size: {'x': 481, 'y': 398}

-S2C_MSIL2A_20260125T111341_N0511_R137_T29TQJ_20260125T145411.zarr
   CRS: EPSG:32629
   Clipped size: {'x': 451, 'y': 432}


### Maybe this is related with x-cube failing?

In [3]:
from xcube.core.store import new_data_store
from xcube.util.config import load_configs
from xcube.webapi.viewer import Viewer
from xcube_resampling.utils import reproject_bbox

In [7]:
store = new_data_store("eopf-zarr")
store.list_data_ids()

['sentinel-2-l1c',
 'sentinel-2-l2a',
 'sentinel-3-olci-l1-efr',
 'sentinel-3-olci-l2-lfr',
 'sentinel-3-slstr-l1-rbt',
 'sentinel-3-slstr-l2-lst']

In [8]:
crs_utm = "EPSG:32630"
bbox_utm = reproject_bbox(bbox_villaviciosa, "EPSG:4326", crs_utm)



In [9]:
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm, # bbox in degrees can also be used
    time_range=temporal_extent,
    spatial_res=10, # in that case convert m to degrees: 10 / 111320 
    crs=crs_utm, # and change to EPSG:4326
    variables=["b01", "b02", "b03", "b04", "scl"],
)
ds

/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251104T112139_N0511_R037_T30TUP_20251104T134759' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251104T112139_N0511_R037_T29TQJ_20251104T134759' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2A_MSIL2A_20251101T112241_N0511_R037_T30TUP_20251101T151319' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2A_MSIL2A_20251101T112241_N0511_R037_T29TQJ_20251101T151319' is deprecated.
  warnings.warn(
/Users/fer/anaconda3/envs/landscape/lib/python3.13/site-packages/pystac/item.py:481: DeprecatedWarning: The item 'S2B_MSIL2A_20251101T111119_N0511_R137_T30TUP_20251101T

FileNotFoundError: No such file or directory: 'https://objects.eodc.eu:443/e05ab01a9d56408d82ac32d69a5aae2a:202509-s02msil2a-eu/25/products/cpm_v256/S2B_MSIL2A_20250925T112109_N0511_R037_T29TQJ_20250925T151904.zarr'